In [1]:
import pandas as pd

In [41]:
df = pd.read_csv("./Dataset/E-commerce.csv", )
print(df.columns)

Index(['Unnamed: 0', 'event_time', 'category_code', 'price', 'user_id',
       'user_session'],
      dtype='object')


In [42]:
df.drop("Unnamed: 0", axis=1,inplace=True)

In [43]:
df.head()

,event_time,category_code,price,user_id,user_session
0,2019-12-01 00:00:02 UTC,computers.peripherals.printer,132.31,535135317,61792a26-672f-4e61-9832-7b63bb1714db
1,2019-12-01 00:00:39 UTC,construction.tools.light,167.29,569958205,80afff78-be88-479a-8930-83b3e6220926
2,2019-12-01 00:01:35 UTC,construction.tools.light,167.03,557794415,6fecf566-ebb0-4e70-a243-cdc13ce044cb
3,2019-12-01 00:01:54 UTC,construction.tools.light,154.41,561593478,cc3328ce-e22a-49ba-b92a-3e143fb4b3d6
4,2019-12-01 00:02:01 UTC,construction.tools.light,227.64,555295228,3de3ac21-f446-4cf5-b3c3-06a051c5caa9


### Category split

In [44]:
#category_code의 sub category들을 '.'으로 구분하여 대분류와 중, 소 분류 구분
df_category_code = df['category_code'].str.split(".", expand=True)
#대분류
category_df = df_category_code[0]
#대분류 병합
df = pd.concat([df,category_df], axis=1)
#column 명 변경
df.rename(columns={0:'category'}, inplace=True)
#기존의 category_code 삭제
df.drop(columns=["category_code"],inplace=True)
df.head()

,event_time,price,user_id,user_session,category
0,2019-12-01 00:00:02 UTC,132.31,535135317,61792a26-672f-4e61-9832-7b63bb1714db,computers
1,2019-12-01 00:00:39 UTC,167.29,569958205,80afff78-be88-479a-8930-83b3e6220926,construction
2,2019-12-01 00:01:35 UTC,167.03,557794415,6fecf566-ebb0-4e70-a243-cdc13ce044cb,construction
3,2019-12-01 00:01:54 UTC,154.41,561593478,cc3328ce-e22a-49ba-b92a-3e143fb4b3d6,construction
4,2019-12-01 00:02:01 UTC,227.64,555295228,3de3ac21-f446-4cf5-b3c3-06a051c5caa9,construction


### Event_time

In [45]:
df['event_time'] = df['event_time'].str[:-4]
df['event_time'] = pd.to_datetime(df['event_time'], format='%Y-%m-%d %H:%M:%S', errors='raise')

# 시(숫자)
df['event_hour'] = df['event_time'].dt.hour 
# 요일숫자(0-월, 1-화)
df['event_weekday'] = df['event_time'].dt.weekday      

### Event hour

In [46]:
hour_names = ['dawn', 'mornig', 'afternoon', 'evening']
df['event_timeline'] = pd.cut(x = df['event_hour'], #데이터 배열
                     bins =4,    #구간 개수
                     labels = hour_names,     #구간명
                     include_lowest = True)  # 첫 경계값 포함여부(구간의 하위값)

In [47]:
df.head()

,event_time,price,user_id,user_session,category,event_hour,event_weekday,event_timeline
0,2019-12-01 00:00:02,132.31,535135317,61792a26-672f-4e61-9832-7b63bb1714db,computers,0,6,dawn
1,2019-12-01 00:00:39,167.29,569958205,80afff78-be88-479a-8930-83b3e6220926,construction,0,6,dawn
2,2019-12-01 00:01:35,167.03,557794415,6fecf566-ebb0-4e70-a243-cdc13ce044cb,construction,0,6,dawn
3,2019-12-01 00:01:54,154.41,561593478,cc3328ce-e22a-49ba-b92a-3e143fb4b3d6,construction,0,6,dawn
4,2019-12-01 00:02:01,227.64,555295228,3de3ac21-f446-4cf5-b3c3-06a051c5caa9,construction,0,6,dawn


### Event day : weekday or weekend

In [48]:
dayofweek_names = ['weekday', 'weekend']
weekend_dins = [0, 4, 6]
df['weekday'] = pd.cut(x = df['event_weekday'], #데이터 배열
                     bins = weekend_dins,    #구간 개수
                     labels = dayofweek_names,     #구간명
                     include_lowest = True)  # 첫 경계값 포함여부(구간의 하위값)

In [49]:
df.head()

,event_time,price,user_id,user_session,category,event_hour,event_weekday,event_timeline,weekday
0,2019-12-01 00:00:02,132.31,535135317,61792a26-672f-4e61-9832-7b63bb1714db,computers,0,6,dawn,weekend
1,2019-12-01 00:00:39,167.29,569958205,80afff78-be88-479a-8930-83b3e6220926,construction,0,6,dawn,weekend
2,2019-12-01 00:01:35,167.03,557794415,6fecf566-ebb0-4e70-a243-cdc13ce044cb,construction,0,6,dawn,weekend
3,2019-12-01 00:01:54,154.41,561593478,cc3328ce-e22a-49ba-b92a-3e143fb4b3d6,construction,0,6,dawn,weekend
4,2019-12-01 00:02:01,227.64,555295228,3de3ac21-f446-4cf5-b3c3-06a051c5caa9,construction,0,6,dawn,weekend


### Event day : day of week

In [50]:
dayofweek_names = ['Mon', 'Tus', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
df['dayofweek'] = pd.cut(x = df['event_weekday'], #데이터 배열
                     bins = 7,    #구간 개수
                     labels = dayofweek_names,     #구간명
                     include_lowest = True)  # 첫 경계값 포함여부(구간의 하위값)

In [51]:
df.head()

,event_time,price,user_id,user_session,category,event_hour,event_weekday,event_timeline,weekday,dayofweek
0,2019-12-01 00:00:02,132.31,535135317,61792a26-672f-4e61-9832-7b63bb1714db,computers,0,6,dawn,weekend,Sun
1,2019-12-01 00:00:39,167.29,569958205,80afff78-be88-479a-8930-83b3e6220926,construction,0,6,dawn,weekend,Sun
2,2019-12-01 00:01:35,167.03,557794415,6fecf566-ebb0-4e70-a243-cdc13ce044cb,construction,0,6,dawn,weekend,Sun
3,2019-12-01 00:01:54,154.41,561593478,cc3328ce-e22a-49ba-b92a-3e143fb4b3d6,construction,0,6,dawn,weekend,Sun
4,2019-12-01 00:02:01,227.64,555295228,3de3ac21-f446-4cf5-b3c3-06a051c5caa9,construction,0,6,dawn,weekend,Sun


### visualization을 위한 csv

In [52]:
df.to_csv("./Dataset/E-commerce(visualization).csv", index=False)

### One-hot encoding

In [53]:
df.iloc[0:10]

,event_time,price,user_id,user_session,category,event_hour,event_weekday,event_timeline,weekday,dayofweek
0,2019-12-01 00:00:02,132.31,535135317,61792a26-672f-4e61-9832-7b63bb1714db,computers,0,6,dawn,weekend,Sun
1,2019-12-01 00:00:39,167.29,569958205,80afff78-be88-479a-8930-83b3e6220926,construction,0,6,dawn,weekend,Sun
2,2019-12-01 00:01:35,167.03,557794415,6fecf566-ebb0-4e70-a243-cdc13ce044cb,construction,0,6,dawn,weekend,Sun
3,2019-12-01 00:01:54,154.41,561593478,cc3328ce-e22a-49ba-b92a-3e143fb4b3d6,construction,0,6,dawn,weekend,Sun
4,2019-12-01 00:02:01,227.64,555295228,3de3ac21-f446-4cf5-b3c3-06a051c5caa9,construction,0,6,dawn,weekend,Sun
5,2019-12-01 00:02:13,334.37,567950899,004cd693-5f13-4e07-85af-6b7d9242798c,furniture,0,6,dawn,weekend,Sun
6,2019-12-01 00:03:24,154.41,561593478,cc3328ce-e22a-49ba-b92a-3e143fb4b3d6,construction,0,6,dawn,weekend,Sun
7,2019-12-01 00:04:39,131.25,567950899,004cd693-5f13-4e07-85af-6b7d9242798c,construction,0,6,dawn,weekend,Sun
8,2019-12-01 00:04:40,384.54,517876634,79428fc0-959b-4a17-8079-e11493c81f48,electronics,0,6,dawn,weekend,Sun
9,2019-12-01 00:04:44,912.50,518271413,daa60361-1340-426c-89ae-fb3568819dd7,construction,0,6,dawn,weekend,Sun


In [54]:
df = pd.get_dummies(df, columns = ['category','event_timeline','weekday',"dayofweek"])

In [55]:
df = df.drop(["event_hour","event_weekday"],axis=1)

In [56]:
df.columns

Index(['event_time', 'price', 'user_id', 'user_session',
       'category_accessories', 'category_apparel', 'category_appliances',
       'category_auto', 'category_computers', 'category_construction',
       'category_country_yard', 'category_electronics', 'category_furniture',
       'category_kids', 'category_medicine', 'category_sport',
       'category_stationery', 'event_timeline_dawn', 'event_timeline_mornig',
       'event_timeline_afternoon', 'event_timeline_evening', 'weekday_weekday',
       'weekday_weekend', 'dayofweek_Mon', 'dayofweek_Tus', 'dayofweek_Wed',
       'dayofweek_Thu', 'dayofweek_Fri', 'dayofweek_Sat', 'dayofweek_Sun'],
      dtype='object')

In [57]:
df[[ 'category_accessories', 'category_apparel', 'category_appliances',
       'category_auto', 'category_computers', 'category_construction',
       'category_country_yard']]

,category_accessories,category_apparel,category_appliances,category_auto,category_computers,category_construction,category_country_yard
0,0,0,0,0,1,0,0
1,0,0,0,0,0,1,0
2,0,0,0,0,0,1,0
3,0,0,0,0,0,1,0
4,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...
6080756,0,0,0,1,0,0,0
6080757,0,0,0,0,0,1,0
6080758,0,0,0,0,0,0,0
6080759,0,0,1,0,0,0,0


In [58]:
df[['category_electronics', 'category_furniture',
       'category_kids', 'category_medicine', 'category_sport',
       'category_stationery']]

,category_electronics,category_furniture,category_kids,category_medicine,category_sport,category_stationery
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
6080756,0,0,0,0,0,0
6080757,0,0,0,0,0,0
6080758,1,0,0,0,0,0
6080759,0,0,0,0,0,0


In [59]:
df[['event_timeline_dawn', 'event_timeline_mornig',
       'event_timeline_afternoon', 'event_timeline_evening', 'weekday_weekday',
       'weekday_weekend']]

,event_timeline_dawn,event_timeline_mornig,event_timeline_afternoon,event_timeline_evening,weekday_weekday,weekday_weekend
0,1,0,0,0,0,1
1,1,0,0,0,0,1
2,1,0,0,0,0,1
3,1,0,0,0,0,1
4,1,0,0,0,0,1
...,...,...,...,...,...,...
6080756,0,0,0,1,1,0
6080757,0,0,0,1,1,0
6080758,0,0,0,1,1,0
6080759,0,0,0,1,1,0


In [60]:
df[['dayofweek_Mon', 'dayofweek_Tus', 'dayofweek_Wed',
       'dayofweek_Thu', 'dayofweek_Fri', 'dayofweek_Sat', 'dayofweek_Sun']]

,dayofweek_Mon,dayofweek_Tus,dayofweek_Wed,dayofweek_Thu,dayofweek_Fri,dayofweek_Sat,dayofweek_Sun
0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...
6080756,0,0,0,0,1,0,0
6080757,0,0,0,0,1,0,0
6080758,0,0,0,0,1,0,0
6080759,0,0,0,0,1,0,0


In [61]:
df.to_csv("./Dataset/E-commerce(one-hot).csv", index=False)